In [5]:
%pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl (198 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.6.15-py3-none-any.whl (157 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]
Note: you may need to restart the kernel to use updated packages.


In [83]:
import pandas as pd
df =pd.read_csv("events.csv")
df

,Event_ID,Event_Name,Event_Date,Outcome,Primary_Type,Secondary_Type,Region,Country_1,Country_2,External_Actors,Key_Individuals,Key_Groups_Entities,Date_Justification,Search_Terms
0,EUR_2022_UKRAINE_CONFLICT,Full-Scale Invasion of Ukraine,2022-02-24,Conflict,Invasion,Interstate War,Europe,Ukraine,Russia,"Belarus,Chinese Hackers,International Criminal...","Putin,Zelenskyy,Biden,Alexander Bortnikov,Niko...","Kremlin,NATO,Russian Army,Ukrainian Army,Donet...",NaN,"Ukraine invasion,Russia special military opera..."
1,MEA_2023_GAZA_CONFLICT,Hamas Attack on Israel,2023-10-07,Conflict,Cross-border Attack,Asymmetric War,Middle East,Israel,Palestine,"Iran,IDF,Gaza,Islamic Jihad","Netanyahu,Sinwar,Mohammed Deif","Hamas,Israel Defense Forces,Al-Qassam Brigades...",NaN,"Hamas attack,Iron Swords,Gaza invasion"
2,AFR_2020_TIGRAY_CONFLICT,Tigray War Start,2020-11-04,Conflict,Civil War,Intervention,Africa,Ethiopia,NaN,"Eritrea,militas from Amhara and Afar","Abiy Ahmed,Debretsion Gebremichael,Getachew Re...","Tigray,TDF,ENDF,Eritrean Defence Forces,TPLF,IRAT",NaN,"Tigray war,Ethiopia conflict,Abiy Ahmed"
3,AFR_2011_LIBYA_CONFLICT,Libyan Civil War Start,2011-02-15,Conflict,Civil War,Foreign Intervention,Africa,Libya,NaN,"NATO,Turkey,UAE,Egypt,France,Wagner Group","Gaddafi,Mustafa Abdul Jalil,Khalifa Haftar,Fay...","National Transitional Council,NTC,UNSC Resolut...",NaN,"Libya civil war,Gaddafi speech,Benghazi uprising"
4,EUR_2008_GEORGIA_CONFLICT,Russo-Georgian War,2008-08-07,Conflict,Invasion,Interstate War,Europe,Georgia,Russia,"USA,NATO,France,Separatist authorities and arm...","Saakashvili,Medvedev,Putin,Borisov,Nicolas Sar...","South Ossetia,Abkhazia,Georgian Armed forces,R...",NaN,"Russo-Georgian war,South Ossetia conflict"
5,MEA_2015_YEMEN_CONFLICT,Saudi-led Intervention in Yemen,2015-03-26,Conflict,Intervention,Proxy War,Middle East,Yemen,Saudi Arabia,"United Arab Emirates, Egypt, Jordan, Morocco, ...","Hadi,King Salman,Mohammed bin Salman,President...","Houthi,Operation Decisive Storm,Saudi-led coal...",NaN,"Saudi intervention Yemen,Houthi conflict"
6,MEA_2011_SYRIA_CONFLICT,Syrian Civil War Begins,2011-03-15,Conflict,Civil War,Proxy War,Middle East,Syria,Saudi Arabia,"Russia,USA,Iran,Turkey,Saudi Arabia,Qatar","Assad,Obama,Abu Mohammed al-Jolani,Putin,Qasse...","Free Syrian Army,FSA,Daraa protests,ISIS,Wagne...",NaN,"Syria civil war,Assad speech,Daraa"
7,EUR_2020_NAGORNO_CONFLICT,Second Nagorno-Karabakh War,2020-09-27,Conflict,Interstate War,NaN,Europe,Armenia,Azerbaijan,"Russia,Turkey,Israel,Syrian mercenaries,Afghan...","Pashinyan,Aliyev,Putin","Nagorno-Karabakh,Artsakh,Armenian Army,Azerbai...",NaN,"Nagorno-Karabakh war,Armenia Azerbaijan conflict"
8,EUR_2014_DONBAS_CONFLICT,Donbas Conflict Start,2014-04-06,Conflict,Hybrid War,Insurgency,Europe,Ukraine,Russia,"OSCE,France,Germany,USA,EU","Turchynov,Strelkov,Viktor Yanukovych,Petro Por...","Donetsk People's Republic,Luhansk People's Rep...",NaN,"Donbas war,Ukraine separatists"
9,EUR_2014_CRIMEA_CONFLICT,Annexation of Crimea,2014-02-27,Conflict,Annexation,Hybrid War,Europe,Ukraine,Russia,"US,EU,NATO,UN,CIS,OSCE","Aksyonov,Putin,Oleksandr Turchynov,Viktor Yanu...","little green men,referendum,Supreme Council of...",NaN,"Crimea annexation,Russia Crimea"


# NEWS DATA COLLECTION

In [ ]:
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import dotenv
dotenv.load_dotenv()

# --- CONFIGURATION & CONSTANTS ---
LEAD_UP_DAYS = 90
DATA_DIR = "raw_data"
EVENTS_FILE = "events.csv"

# --- API HELPERS ---

def build_query(event):
    """Builds a robust boolean query for the APIs."""
    
    # 1. Country terms (must have at least one)
    country_terms = f'({event["Country_1"]}'
    if pd.notna(event["Country_2"]):
        country_terms += f' OR {event["Country_2"]}'
    country_terms += ')'

    # 2. Key entities (should have at least one)
    actors = []
    if pd.notna(event["Key_Individuals"]):
        actors.extend([f'{name.strip()}' for name in event["Key_Individuals"].split(',')])
    if  pd.notna(event["Key_Groups_Entities"]):
         actors.extend([f'{name.strip()}' for name in event["Key_Groups_Entities"].split(',')])
    
    actor_terms = " AND (" + " OR ".join(actors) + ")" if actors else ""

    # 3. Thematic terms (from the Search_Terms column)
    theme_terms = ""
    if  pd.notna(event["Search_Terms"]):
        themes = [f'{term.strip()}' for term in event["Search_Terms"].split(',')]
        theme_terms = " AND (" + " OR ".join(themes) + ")"

    # Combine all parts
    full_query = country_terms + actor_terms + theme_terms
    return full_query

def fetch_guardian_data(query, start_date, end_date):
    """Fetches data from The Guardian API, handling pagination."""
    print("  -> Fetching from The Guardian...")
    articles = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        params = {
            'q': query,
            'from-date': start_date,
            'to-date': end_date,
            'api-key': os.getenv('GUARDIAN_API_KEY'),
            'page-size': 50,
            'page': page,
            'show-fields': 'bodyText'
        }
        try:
            response = requests.get('https://content.guardianapis.com/search', params=params)
            response.raise_for_status()
            data = response.json()['response']
            
            for item in data.get('results', []):
                articles.append({
                    'source': 'The Guardian',
                    'date': item['webPublicationDate'],
                    'headline': item['webTitle'],
                    'snippet': item.get('fields', {}).get('bodyText', '')[:500] # Get first 500 chars of body - why?
                })
            
            total_pages = data.get('pages', 1)
            print(f"     Got page {page}/{total_pages}...")
            page += 1
            time.sleep(1) # IMPORTANT: Respect API rate limits
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from The Guardian: {e}")
            break
            
    return articles

def build_nyt_query_and_filter(event):
    """
    Builds a more flexible query and a separate filter for the NYT.
    Returns a tuple: (query, filter_query)
    """
    # 1. The Filter Query (fq): Highly reliable for location
    # We use the 'glocations' field in the NYT data.
    glocations = []
    if pd.notna(event["Country_1"]):
        glocations.append(f'"{event["Country_1"].upper()}"')
    if pd.notna(event["Country_2"]):
        glocations.append(f'"{event["Country_2"].upper()}"')
    filter_query = f'glocations:({" OR ".join(glocations)})' if glocations else None

    # 2. The Keyword Query (q): A flexible search for actors and themes
    keywords = []
    if pd.notna(event["Search_Terms"]):
        keywords.extend([f'"{term.strip()}"' for term in event["Search_Terms"].split(',')])
    if pd.notna(event["Key_Groups_Entities"]):
        # Add the first 2 key groups for relevance
        keywords.extend([f'"{term.strip()}"' for term in event["Key_Groups_Entities"].split(',')[:2]])
        
    keyword_query = " OR ".join(keywords) if keywords else event['Event_Name'] # Fallback to event name
    
    return keyword_query, filter_query



In [88]:
event_date = datetime.strptime(df.loc[7]['Event_Date'], '%Y-%m-%d')
start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)

start_date_str = start_datetime.strftime('%Y-%m-%d')
end_date_str = event_date.strftime('%Y-%m-%d')
articles = fetch_guardian_data(build_nyt_query_and_filter(df.loc[7])[0], start_date_str, end_date_str)
articles

  -> Fetching from The Guardian...
     Got page 1/1...


[{'source': 'The Guardian',
  'date': '2020-09-27T10:25:21Z',
  'headline': 'At least 16 dead in Armenia-Azerbaijan clashes over disputed region',
  'snippet': 'At least 16 troops and civilians have been killed in the worst fighting between Azerbaijani and Armenian forces since 2016, raising fears of instability in a region that provides crucial transit routes for gas and oil to the international market. Armenia declared martial law and ordered the total mobilisation of its military on Sunday after claiming to have destroyed several Azerbaijani aircraft and tanks in early-morning clashes. It accused Azerbaijan of carrying out air and artillery attacks o'},
 {'source': 'The Guardian',
  'date': '2020-07-26T05:31:15Z',
  'headline': 'Moscow’s fruit stalls become frontline of a border skirmish  2,000 miles away',
  'snippet': 'When Saribek Gevorkyan heard reports that Food City, a vast food distribution centre owned by entrepreneurs from Azerbaijan, had suddenly blocked Armenian farmers a

In [67]:
len(articles)

836

In [65]:
start_date_nyt = start_datetime.strftime('%Y%m%d')
end_date_nyt = event_date.strftime('%Y%m%d')
data =fetch_nyt_data(build_nyt_query_and_filter(df.loc[0]), start_date_nyt, end_date_nyt)

  -> Fetching from The New York Times...


In [59]:
params = {
'q': build_nyt_query(df.loc[0]),
'begin_date': start_date_nyt,
'end_date': end_date_nyt,
'api-key': os.getenv('NYT_API_KEY'),
'page': 0
}
response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params=params)
data = response.json()['response']
docs = data.get('docs', [])
docs

[{'abstract': 'If the West fails to meet its security demands, Moscow could take measures like placing nuclear missiles close to the U.S. coastline, Russian officials have hinted.',
  'byline': {'original': 'By Anton Troianovski and David E. Sanger'},
  'document_type': 'article',
  'headline': {'main': 'Russia Issues Subtle Threats More Far-Reaching Than a Ukraine Invasion',
   'kicker': '',
   'print_headline': 'Putin Could Cause Trouble Not Just in Ukraine but in West, Too'},
  '_id': 'nyt://article/73e801a9-d932-5899-96ec-11d092bbb45e',
  'keywords': [{'name': 'Location', 'value': 'Russia', 'rank': 1},
   {'name': 'Location', 'value': 'Ukraine', 'rank': 2},
   {'name': 'Subject', 'value': 'Defense and Military Forces', 'rank': 3},
   {'name': 'Subject', 'value': 'Cyberwarfare and Defense', 'rank': 4},
   {'name': 'Organization',
    'value': 'North Atlantic Treaty Organization',
    'rank': 5},
   {'name': 'Subject',
    'value': 'United States International Relations',
    'rank':

In [57]:
len(docs)

10

In [68]:
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import dotenv
dotenv.load_dotenv()

# --- CONFIGURATION & CONSTANTS ---
LEAD_UP_DAYS = 90
DATA_DIR = "raw_data"
EVENTS_FILE = "events.csv"

# --- API HELPERS ---

def build_query(event):
    """Builds a robust boolean query for the APIs."""
    
    # 1. Country terms (must have at least one)
    country_terms = f'({event["Country_1"]}'
    if pd.notna(event["Country_2"]):
        country_terms += f' OR {event["Country_2"]}'
    country_terms += ')'

    # 2. Key entities (should have at least one)
    actors = []
    if pd.notna(event["Key_Individuals"]):
        actors.extend([f'{name.strip()}' for name in event["Key_Individuals"].split(',')])
    if  pd.notna(event["Key_Groups_Entities"]):
         actors.extend([f'{name.strip()}' for name in event["Key_Groups_Entities"].split(',')])
    
    actor_terms = " AND (" + " OR ".join(actors) + ")" if actors else ""

    # 3. Thematic terms (from the Search_Terms column)
    theme_terms = ""
    if  pd.notna(event["Search_Terms"]):
        themes = [f'{term.strip()}' for term in event["Search_Terms"].split(',')]
        theme_terms = " AND (" + " OR ".join(themes) + ")"

    # Combine all parts
    full_query = country_terms + actor_terms + theme_terms
    return full_query

def fetch_guardian_data(query, start_date, end_date):
    """Fetches data from The Guardian API, handling pagination."""
    print("  -> Fetching from The Guardian...")
    articles = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        params = {
            'q': query,
            'from-date': start_date,
            'to-date': end_date,
            'api-key': os.getenv('GUARDIAN_API_KEY'),
            'page-size': 50,
            'page': page,
            'show-fields': 'bodyText'
        }
        try:
            response = requests.get('https://content.guardianapis.com/search', params=params)
            response.raise_for_status()
            data = response.json()['response']
            
            for item in data.get('results', []):
                articles.append({
                    'source': 'The Guardian',
                    'date': item['webPublicationDate'],
                    'headline': item['webTitle'],
                    'snippet': item.get('fields', {}).get('bodyText', '')[:500] # Get first 500 chars of body - why?
                })
            
            total_pages = data.get('pages', 1)
            print(f"     Got page {page}/{total_pages}...")
            page += 1
            time.sleep(1) # IMPORTANT: Respect API rate limits
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from The Guardian: {e}")
            break
            
    return articles


In [70]:
"""Main function to orchestrate the data acquisition process."""
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

events_df = pd.read_csv(EVENTS_FILE)

for _, event in events_df.iterrows():
    event_id = event['Event_ID']
    output_filename = os.path.join(DATA_DIR, f"{event_id}.csv")

    if os.path.exists(output_filename):
        print(f"Skipping {event_id}: Data file already exists.")
        continue

    print(f"\nProcessing Event: {event_id} - {event['Event_Name']}")

    # Prepare dates and query
    event_date = datetime.strptime(event['Event_Date'], '%Y-%m-%d')
    start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)
    
    start_date_str = start_datetime.strftime('%Y-%m-%d')
    end_date_str = event_date.strftime('%Y-%m-%d')
    
    query = build_query(event)
    print(f"  -> Query: {query[:200]}...") # Print first 200 chars of query

    # Fetch data from all sources
    guardian_articles = fetch_guardian_data( query, start_date_str, end_date_str)

    # Combine and save
    all_articles = guardian_articles 
    if all_articles:
        df = pd.DataFrame(all_articles)
        df.to_csv(output_filename, index=False, encoding='utf-8')
        print(f"Success! Saved {len(df)} articles to {output_filename}")
    else:
        print(f"Warning: No articles found for {event_id}. An empty file will not be created.")



Processing Event: EUR_2022_UKRAINE_CONFLICT - Full-Scale Invasion of Ukraine
  -> Query: (Ukraine OR Russia) AND (Putin OR Zelenskyy OR Biden OR Alexander Bortnikov OR Nikolai Patrushev OR Yury Kovalchuk OR Andrii Sybiha OR Mark Rutte OR Kremlin OR NATO OR Russian Army OR Ukrainian Army O...
  -> Fetching from The Guardian...
     Got page 1/17...
     Got page 2/17...
     Got page 3/17...
     Got page 4/17...
     Got page 5/17...
     Got page 6/17...
     Got page 7/17...
     Got page 8/17...
     Got page 9/17...
     Got page 10/17...
     Got page 11/17...
     Got page 12/17...
     Got page 13/17...
     Got page 14/17...
     Got page 15/17...
     Got page 16/17...
     Got page 17/17...
Success! Saved 836 articles to raw_data/EUR_2022_UKRAINE_CONFLICT.csv

Processing Event: MEA_2023_GAZA_CONFLICT - Hamas Attack on Israel
  -> Query: (Israel OR Palestine) AND (Netanyahu OR Sinwar OR Mohammed Deif OR Hamas OR Israel Defense Forces OR Al-Qassam Brigades OR Lions' Den OR PFL

In [89]:
for event in events_df.iterrows():
    #print(event['Event_ID'],len(pd.read_csv(f"raw_data/{event['Event_ID']}.csv")))
    try:
        print(event[1]['Event_ID'],len(pd.read_csv(f"raw_data/{event[1]['Event_ID']}.csv")))
    except:
        print(event[1]['Event_ID'],"not found- bad query- skipping,(barely enough data )")
    

EUR_2022_UKRAINE_CONFLICT 836
MEA_2023_GAZA_CONFLICT 84
AFR_2020_TIGRAY_CONFLICT 10
AFR_2011_LIBYA_CONFLICT 69
EUR_2008_GEORGIA_CONFLICT 23
MEA_2015_YEMEN_CONFLICT 425
MEA_2011_SYRIA_CONFLICT 80
EUR_2020_NAGORNO_CONFLICT not found- bad query- skipping,(barely enough data )
EUR_2014_DONBAS_CONFLICT 886
EUR_2014_CRIMEA_CONFLICT 174
EAS_2017_KOREA_NOCONFLICT 624
MEA_2020_IRAN_NOCONFLICT 278
EUR_2021_UKRAINE_NOCONFLICT 157
SAS_2020_INDOCHINA_NOCONFLICT 341
EUR_2015_TURKEYRUSSIA_NOCONFLICT 642
SAS_2019_PULWAMA_NOCONFLICT 218
EAS_2012_SCARBOROUGH_NOCONFLICT 112
AFR_2012_MALI_CONFLICT 6
SAS_2025_PHALGAMA_NOCONFLICT 70


# PRUING and Processing raw_data
we'll take a random sample of 500 records from events that have over 500 articles,
those that don't we'll take all of them

### reasoning:
- when we have a huge imbalance in the volume of data we obtain for different events, it becomes increasingly difficult for the model to effectively train on the minority class.
- a small example: if you're training a model to detect cats and elephands, and you have 1000 cat images and 100 elephant images, the model will be biased towards cats and will perform poorly on elephant detection; it'll think the elephant is a fat mutated cat- not ideal,

it may seem like taking we're throwing away alot of data, and on the flipside, the disparity still exists, but, this is a tradeoff we're willing to make for the sake of model performance.

In [91]:
import os
import pandas as pd

# --- CONFIGURATION ---
RAW_DATA_DIR = "raw_data"
PROCESSED_DATA_DIR = "processed_data"
ARTICLE_CAP = 500  # The maximum number of articles we'll use per event
RANDOM_STATE = 42  # Ensures our random sample is the same every time we run it

def balance_data():
    """
    Reads all raw CSVs, applies a cap to balance the dataset,
    and saves the results to a new directory.
    """
    if not os.path.exists(PROCESSED_DATA_DIR):
        os.makedirs(PROCESSED_DATA_DIR)

    raw_files = [f for f in os.listdir(RAW_DATA_DIR) if f.endswith('.csv')]
    
    if not raw_files:
        print(f"Error: No CSV files found in the '{RAW_DATA_DIR}' directory.")
        return

    print("Balancing dataset using the 'capping' strategy...")
    total_raw_articles = 0
    total_processed_articles = 0

    for filename in raw_files:
        print(f"  -> Processing {filename}...")
        
        # Read the raw data
        raw_filepath = os.path.join(RAW_DATA_DIR, filename)
        df = pd.read_csv(raw_filepath)
        
        num_articles = len(df)
        total_raw_articles += num_articles

        # Apply the capping logic
        if num_articles > ARTICLE_CAP:
            print(f"     Event has {num_articles} articles. Capping to {ARTICLE_CAP}.")
            # Take a reproducible random sample
            processed_df = df.sample(n=ARTICLE_CAP, random_state=RANDOM_STATE)
        else:
            print(f"     Event has {num_articles} articles. Using all of them.")
            processed_df = df
        
        total_processed_articles += len(processed_df)
        
        # Save the new balanced dataframe
        processed_filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        processed_df.to_csv(processed_filepath, index=False, encoding='utf-8')

    print("\n--- Balancing Complete ---")
    print(f"Total articles before balancing: {total_raw_articles}")
    print(f"Total articles after balancing: {total_processed_articles}")
    print(f"Balanced data saved to the '{PROCESSED_DATA_DIR}' folder.")


balance_data()

Balancing dataset using the 'capping' strategy...
  -> Processing MEA_2011_SYRIA_CONFLICT.csv...
     Event has 80 articles. Using all of them.
  -> Processing MEA_2015_YEMEN_CONFLICT.csv...
     Event has 425 articles. Using all of them.
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT.csv...
     Event has 341 articles. Using all of them.
  -> Processing SAS_2019_PULWAMA_NOCONFLICT.csv...
     Event has 218 articles. Using all of them.
  -> Processing AFR_2011_LIBYA_CONFLICT.csv...
     Event has 69 articles. Using all of them.
  -> Processing EAS_2017_KOREA_NOCONFLICT.csv...
     Event has 624 articles. Capping to 500.
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT.csv...
     Event has 112 articles. Using all of them.
  -> Processing EUR_2014_DONBAS_CONFLICT.csv...
     Event has 886 articles. Capping to 500.
  -> Processing MEA_2020_IRAN_NOCONFLICT.csv...
     Event has 278 articles. Using all of them.
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT.csv...
     Event has 70 articles. U

## FEATURE ENGINEERING



In [90]:
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [92]:
import os
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# --- CONFIGURATION ---
PROCESSED_DATA_DIR = "processed_data"
FEATURES_DIR = "features"

def analyze_sentiment():
    """
    Reads all processed CSVs, calculates sentiment for each article,
    and saves the results to a new 'features' directory.
    """
    if not os.path.exists(FEATURES_DIR):
        os.makedirs(FEATURES_DIR)

    processed_files = [f for f in os.listdir(PROCESSED_DATA_DIR) if f.endswith('.csv')]
    
    if not processed_files:
        print(f"Error: No CSV files found in the '{PROCESSED_DATA_DIR}' directory.")
        return

    # Initialize the VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    print("Starting sentiment analysis...")

    for filename in processed_files:
        print(f"  -> Processing {filename}...")
        
        # Read the processed data
        filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        df = pd.read_csv(filepath)
        
        # Prepare the text for analysis by combining headline and snippet
        # We use .fillna('') to handle any potentially empty fields gracefully
        df['text_to_analyze'] = df['headline'].fillna('') + ' ' + df['snippet'].fillna('')

        # Define a function to get the compound sentiment score
        def get_sentiment_score(text):
            # The 'compound' score is a single, normalized score from -1 to +1
            return analyzer.polarity_scores(text)['compound']

        # Apply the function to our text column to create the new feature
        # This will calculate the sentiment for every row in the dataframe
        df['sentiment_compound'] = df['text_to_analyze'].apply(get_sentiment_score)
        
        # We can drop the temporary analysis column
        df = df.drop(columns=['text_to_analyze'])
        
        # Save the new dataframe with the sentiment feature
        features_filepath = os.path.join(FEATURES_DIR, filename)
        df.to_csv(features_filepath, index=False, encoding='utf-8')

    print("\n--- Feature Engineering: Sentiment Analysis Complete ---")
    print(f"New files with sentiment scores saved to the '{FEATURES_DIR}' folder.")

analyze_sentiment()

Starting sentiment analysis...
  -> Processing MEA_2011_SYRIA_CONFLICT.csv...
  -> Processing MEA_2015_YEMEN_CONFLICT.csv...
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT.csv...
  -> Processing SAS_2019_PULWAMA_NOCONFLICT.csv...
  -> Processing AFR_2011_LIBYA_CONFLICT.csv...
  -> Processing EAS_2017_KOREA_NOCONFLICT.csv...
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT.csv...
  -> Processing EUR_2014_DONBAS_CONFLICT.csv...
  -> Processing MEA_2020_IRAN_NOCONFLICT.csv...
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT.csv...
  -> Processing EUR_2021_UKRAINE_NOCONFLICT.csv...
  -> Processing EUR_2014_CRIMEA_CONFLICT.csv...
  -> Processing EUR_2015_TURKEYRUSSIA_NOCONFLICT.csv...
  -> Processing EUR_2022_UKRAINE_CONFLICT.csv...
  -> Processing MEA_2023_GAZA_CONFLICT.csv...

--- Feature Engineering: Sentiment Analysis Complete ---
New files with sentiment scores saved to the 'features' folder.


In [95]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

KEYWORD_TIERS = {
    1: ['tensions', 'dispute', 'protest', 'unrest', 'sanctions', 'diplomat', 'condemn', 'concern', 'warns'],
    3: ['threat', 'mobilize', 'mobilization', 'mobilizing', 'drill', 'exercise', 'border', 'violation', 'standoff', 'brink'],
    5: ['ultimatum', 'attack', 'airstrike', 'invasion', 'casualties', 'clashes', 'imminent', 'offensive', 'shelling']
}
CERTAINTY_KEYWORDS = {
    1: ['will', 'confirms', 'is', 'fact', 'order', 'declares', 'announces', 'instructs', 'guarantees', 'unquestionably'],
   -1: ['could', 'may', 'might', 'suggests', 'appears', 'reportedly', 'seems', 'potential', 'unconfirmed', 'possibly', 'perhaps']
}
analyzer = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    # The 'compound' score is a single, normalized score from -1 to +1
    return analyzer.polarity_scores(text)['compound']
def calculate_rhetoric_score(text):
    score = 0
    # Convert text to lower case for case-insensitive matching
    lower_text = text.lower()
    
    # Check for keywords from each tier and add points
    for points, keywords in KEYWORD_TIERS.items():
        for keyword in keywords:
            if keyword in lower_text:
                score += points
    return score
def calculate_certainty_score(text):
    score = 0
    lower_text = text.lower()
    for points, keywords in CERTAINTY_KEYWORDS.items():
        for keyword in keywords:
            # We use split() to match whole words to avoid 'is' matching in 'this'
            if keyword in lower_text.split():
                score += points
    return score

def create_rhetoric_score():
    if not os.path.exists(FEATURES_DIR):
        os.makedirs(FEATURES_DIR)

    processed_files = [f for f in os.listdir(PROCESSED_DATA_DIR) if f.endswith('.csv')]
    
    if not processed_files:
        print(f"Error: No CSV files found in the '{PROCESSED_DATA_DIR}' directory.")
        return

    # Initialize the VADER sentiment analyzer
    sentiment_analyzer = SentimentIntensityAnalyzer()

    print("Starting feature engineering...")

    for filename in processed_files:
        print(f"  -> Processing {filename}...")
        
        filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        df = pd.read_csv(filepath)
        
        # Prepare the text for analysis (handles empty snippets)
        df['text_to_analyze'] = df['headline'].fillna('') + ' ' + df['snippet'].fillna('')

        df['sentiment_compound'] = df['text_to_analyze'].apply(get_sentiment_score)
        df['rhetoric_score'] = df['text_to_analyze'].apply(calculate_rhetoric_score)
        df['certainty_score'] = df['text_to_analyze'].apply(calculate_certainty_score)
        
        # We can drop the temporary analysis column
        df = df.drop(columns=['text_to_analyze'])
        features_filepath = os.path.join(FEATURES_DIR, filename)
        df.to_csv(features_filepath, index=False, encoding='utf-8')
    
create_rhetoric_score()


Starting feature engineering...
  -> Processing MEA_2011_SYRIA_CONFLICT.csv...
  -> Processing MEA_2015_YEMEN_CONFLICT.csv...
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT.csv...
  -> Processing SAS_2019_PULWAMA_NOCONFLICT.csv...
  -> Processing AFR_2011_LIBYA_CONFLICT.csv...
  -> Processing EAS_2017_KOREA_NOCONFLICT.csv...
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT.csv...
  -> Processing EUR_2014_DONBAS_CONFLICT.csv...
  -> Processing MEA_2020_IRAN_NOCONFLICT.csv...
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT.csv...
  -> Processing EUR_2021_UKRAINE_NOCONFLICT.csv...
  -> Processing EUR_2014_CRIMEA_CONFLICT.csv...
  -> Processing EUR_2015_TURKEYRUSSIA_NOCONFLICT.csv...
  -> Processing EUR_2022_UKRAINE_CONFLICT.csv...
  -> Processing MEA_2023_GAZA_CONFLICT.csv...


## What we've done so far

Features:
- raw sentiment analysis using vader (0-1 score)
- conflict specific rhetoric score
- certainty score

aside from vader which uses a pre-trained model, 
the remaining features are built using simple principle: assign weights to words based on their implied meaning in this context.

# Aggregating news reports into weekly buckets

doing so will smoothen the data- effectivley getting rid of noise and outliers. 

In [ ]:
import os
import pandas as pd

# --- CONFIGURATION ---
FEATURES_DIR = "features"
# Make sure this points to your final, pruned events file
EVENTS_FILE = "events.csv" 
FINAL_OUTPUT_FILE = "final_model_data.csv"

def aggregate_weekly_features():
    """
    Reads all feature files, aggregates them into weekly bins for each event,
    and saves a single, model-ready CSV file.
    """
    if not os.path.exists(FEATURES_DIR):
        print(f"Error: Directory '{FEATURES_DIR}' not found. Please run feature_engineering.py first.")
        return

    try:
        events_df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError:
        print(f"Error: Events file '{EVENTS_FILE}' not found. Please double-check the filename.")
        return

    all_event_data = []
    feature_files = [f for f in os.listdir(FEATURES_DIR) if f.endswith('.csv')]
    print("Starting weekly aggregation...")

    for filename in feature_files:
        event_id = filename.replace('.csv', '')
        print(f"  -> Aggregating {event_id}...")
        
        filepath = os.path.join(FEATURES_DIR, filename)
        df = pd.read_csv(filepath)

        # Convert date column to datetime objects
        df['date'] = pd.to_datetime(df['date'])
        
        # --- THE FIX IS HERE ---
        # Make the 'date' column timezone-naive to match the event_date
        df['date'] = df['date'].dt.tz_localize(None) # <-- ADD THIS LINE

        # Get the event date from our main events file
        event_info = events_df[events_df['Event_ID'] == event_id]
        if event_info.empty:
            print(f"     Warning: No event info found for {event_id} in {EVENTS_FILE}. Skipping.")
            continue
        
        event_date = pd.to_datetime(event_info['Event_Date'].iloc[0])

        # --- Calculate "Week Before Event" ---
        df['days_before_event'] = (event_date - df['date']).dt.days
        df['week_before_event'] = -(df['days_before_event'] // 7)

        # Filter to only include the 12-week lead-up
        df = df[(df['week_before_event'] >= -12) & (df['week_before_event'] <= -1)]
        
        if df.empty:
            print(f"     Warning: No articles found within the 12-week lead-up for {event_id}.")
            continue

        aggregations = {
            'sentiment_compound': ['mean', 'max', 'min', 'std'],
            'rhetoric_score': ['mean', 'max', 'sum'],
            'certainty_score': ['mean', 'sum'],
            'headline': ['count'] 
        }

        weekly_df = df.groupby('week_before_event').agg(aggregations).reset_index()
        
        weekly_df.columns = ['_'.join(col).strip() for col in weekly_df.columns.values]
        weekly_df = weekly_df.rename(columns={'week_before_event_': 'week_before_event', 'headline_count': 'article_count'})

        weekly_df['Event_ID'] = event_id
        weekly_df['Outcome'] = event_info['Outcome'].iloc[0]
        
        all_event_data.append(weekly_df)

    if not all_event_data:
        print("\nError: No data was aggregated. Please check your input files and paths.")
        return

    final_df = pd.concat(all_event_data, ignore_index=True)

    id_cols = ['Event_ID', 'Outcome', 'week_before_event', 'article_count']
    feature_cols = sorted([col for col in final_df.columns if col not in id_cols])
    final_df = final_df[id_cols + feature_cols]

    final_df.to_csv(FINAL_OUTPUT_FILE, index=False, encoding='utf-8')

    print("\n--- Aggregation Complete ---")
    print(f"Final model-ready data saved to '{FINAL_OUTPUT_FILE}'")
    print(f"The final dataset has {len(final_df)} rows.")

if __name__ == "__main__":
    aggregate_weekly_features()

Error: Events file 'eventscsv' not found. Please double-check the filename.
